---
title: "Research Methods II"
subtitle: "Session 2: MLE & Limited Dependent Variables"
author: Fernando Rios-Avila
format: 
  revealjs: 
    slide-number: true
    width: 1600
    height: 900
    code-fold: true
    echo: true
    css: styles.css 
    highlight-style: github
execute: 
  freeze: auto    
---


## Introduction to Maximum Likelihood Estimation

- This is something we have seen before. 
- MLE is a method to estimate parameters of a model.
  - It can be used to estimate paramaters of linear and nonlinear models
- The idea is to find the values of the parameters that maximize the likelihood function.
  - But what does it mean?

> The likelihood function is the probability of observing the data given the parameters of the model.

So, MLE tries to maximize that probability, under the assumption that we know the distribution of the data.

In other words, we try to identify distributions! (not only conditional mean functions)

# Example


In [ ]:
*| echo: false
*| output: false
%%set
graph_width = default
graph_height = default

## Data

In [ ]:
*| echo: false
*| fig-align: center
clear
set scheme white2
color_style tableau
set seed 1
qui:set obs 1000
gen x1 = rnormal(-1,2)
gen x2 = rnormal(1,1)
gen y = x1 + (x2-x1)*(runiform()<.5)
kdensity y, ysize(6) xsize(10)

## MLE estimation

- To identify the parameters of the model, we need to impose assumptions about the distribution of the data.
- For simplicitly, lets make the assumption that the data is normally distributed.

- The likelihood function for a single observation is:

$$L_i(\mu,\sigma) = \frac{1}{\sigma\sqrt{2\pi}} e^{ -\frac{1}{2}\left(\frac{y_i-\mu}{\sigma}\right)^2 }$$

- And under independent observations assumptions, the Likelihood function for the sample is:

$$LL(\mu,\sigma) = \prod_{i=1}^n L_i(\mu,\sigma)$$

## Graphical representation  


In [ ]:
*| echo: false
*| output: false
mata
my = rangen(-3.6,2.5,50)
sy = rangen(1,2.5,50)
msy=my#J(50,1,1),J(50,1,1)#sy
y  = st_data(.,"y")

ll = J(rows(msy),1,0)
for(i=1;i<=rows(msy);i++){
    ll[i]=sum(lnnormalden(y,msy[i,1],msy[i,2]))
}
fdt = msy, ll
end
getmata fdt*=fdt, force
sum fdt3
*gen efdt = (fdt3 - r(mean)/r(sd)
xtile qefdt=fdt3, n(25)
mscatter fdt1 fdt2, over(qefdt) legend(off) aspect(1) colorpalette(viridis) msymbol(S) msize(1.5) ///
ytitle(Mean) xtitle(SD) xsize(6) ysize(6)
graph export "s2_fig1.png", replace width(1000)

![](s2_fig1.png)

## How Good we did?


In [ ]:
*| echo: false
*| fig-align: center
kdensity y, ysize(6) xsize(10) normal

# LR as an MLE

- We already know that LR can be easily estimated using OLS.
$$\beta = (X'X)^{-1}X'y$$

- But we can also estimate it using MLE.

- Consider the following model:
$$Y_i = \beta_0 + \beta_1 X_{1i} + \beta_2 X_{2i} + \epsilon_i$$

- To estimate the parameters using MLE, we need to make assumptions about the distribution of the error term or the dependent variable.

$$Y_i \sim N(x_i'\beta, \sigma^2)$$

##

- Under that assumption, the likelihood function for a single observation is:

$$L_i(\beta,\sigma) = \frac{1}{\sigma\sqrt{2\pi}} e^{ -\frac{1}{2}\left(\frac{y_i-x_i'\beta}{\sigma}\right)^2 }$$

- Which can be used to construct the MLE estimator for OLS.

- Plot-twist: The MLE estimator for OLS is the same as the OLS estimator.

# Limited Dependent Variables

## Limited Dependent Variables

- Limited dependent variables are variables that are limited in their range of values.
  - For example, binary variables, or variables that are bounded between 0 and 1.
  - Or variables that are bounded between 0 and some positive number.
  - Or variables bounded to take only positive values.
  - etc

- Very Special Case: Endogenous Sample Selection 
  - Looks unbounded, but you only observe a subset of the population.

## Binary Data: LPM/Probit/logit 

- Probit and logit are two models that are used to model binary dependent variables. (Dummies)
  - You can also use OLS (LPM), but has drawbacks
  - You also need to make sure your Dependent Variable is binary!

- When your Dep variable is binary, your goal is determine the probability of observing a 1 (success) of something to happen given a set of covariates.

$$P(y_i=1|x_i) = G(x_i'\beta)$$

The choice of $G$ is what makes the difference between LPM, a probit and logit.

## Probit/Logit

:::: {.columns}
::: {.column}

**LOGIT**

$$G(Z) = \frac{e^{Z}}{1+e^{Z}} = \Lambda(Z)$$

:::
::: {.column}

**Probit**

$$G(Z) = \int_{-\infty}^z \phi(v) dv = \Phi(Z)$$
:::

::::

- Both make sure that $0\leq G(Z) \leq 1$, which doesnt happen with LPM ($G(Z)=Z$)

- And with this, we can use MLE to estimate the parameters of the model.

$$LL(\beta) = \prod_{i=1}^n G(x_i'\beta)^{y_i} (1-G(x_i'\beta))^{1-y_i}$$

## 

One could also think of the probit and logit as a transformation of a latent variable $Y^*$.

$$Y^*_i = x_i'\beta + \epsilon_i$$

- The latent variable is not observed. However, when $Y^*_i>0$, we observe $Y_i=1$.

Here the probabilty of observing a $Y_i=1$ is:

$$\begin{aligned}
P(y_i=1|x_i) &= P(y^*_i>0|x_i) = P(x_i'\beta + \epsilon_i>0|x_i) \\
  &= P( \epsilon_i>-x_i'\beta |x_i) = 1-P( \epsilon_i<-x_i'\beta |x_i) \\
  &= 1-G(-x_i'\beta) 
\end{aligned}
$$

And if $G'$ is symetrical (logit/probit/lpm):

$$P(y_i=1|x_i) = G(x_i'\beta)$$

## Marginal Effects and testing

- LPM estimates can be interpreted Directly as the change in P(y=1|X)
- For Logit and probit, we need to compute the marginal effects.

$$P(y_i=1|x_i) = G(x_i'\beta)$$

$$\frac{\partial P(y_i=1|x_i)}{\partial x_{ij}} = g(x_i'\beta)\beta_j$$

- For testing, 
  - You can use the t-test (or z-test for logit/probit) for coefficients or marginal effects
  - Or use LR test for joint significance of a set of coefficients.
  
$$LR = 1- 2 (LL_ur - LL_r) \sim \chi^2_q$$  

## Example `Stata` {.scrollable}

Load the data


In [ ]:
*| echo: true
*| code-fold: false
*| classes: larger
webuse nhanes2d, clear
des highbp height weight age female
sum   highbp height weight age female i.race [w=finalwgt]

Estimate mode: LPM using weights


In [ ]:
*| echo: true
*| code-fold: false
*| classes: larger
reg highbp height weight age female i.race [pw=finalwgt]
* or svy: reg highbp height weight age female i.race 

Estimate mode: Logit using weights


In [ ]:
*| echo: true
*| code-fold: false
*| classes: larger
logit highbp height weight age female i.race [pw=finalwgt]
* or svy: reg highbp height weight age female i.race 

Joint significance test


In [ ]:
*| echo: true
*| code-fold: false
*| classes: larger
test 2.race 3.race

Marginal Effects: You need to use `margins` command


In [ ]:
*| echo: true
*| code-fold: false
*| classes: larger
margins, dydx(*)

Predicted Probabilities


In [ ]:
*| echo: true
*| output: false
*| code-fold: false
*| classes: larger
predict pr_hat
histogram pr_hat  
graph export s2fig2.png, replace width(1000)

From here, we could also predict HighBP


In [ ]:
*| echo: true
*| code-fold: false
*| classes: larger
gen dpr_hat = pr_hat>.5
tab dpr_hat highbp [w=finalwgt]

# Tobit

## Tobit

- Tobit models are to analyze data with censored information.

- Censored data means that the data is there... but you dont know the exact value.

- For example, if you have data on income, but you only know that some people earn less than 10K, but you dont know how much less.

- The fact that you can see the data, even if you do not know the exact value, helps you to estimate the parameters of the model.

## Visualizing the problem 


In [ ]:
*| echo: false
*| output: false
clear
set obs 1000
gen x = rnormal()

gen y = 1 + x + rnormal()
gen yp = 1 + x 
gen yc = max(y,0)

scatter y x , name(m1, replace) title(Fully observed) ytitle("")  
scatter yc x, name(m2, replace) title(Censored Data)  ytitle("")  

graph combine m1 m2, ycommon
graph export s2fig3.png, replace width(1000)

two (scatter  y x, color(%50)) || (lfit y x) || (line yp x, sort lw(2) lc(gs2%20)), legend(off) name(m1, replace) title(Fully observed)

two (scatter  yc x, color(%50)) || (lfit yc x) || (line yp x, sort lw(2) lc(gs2%20)), legend(off) name(m2, replace) title(Censored Data)

graph combine m1 m2, ycommon
graph export s2fig4.png, replace width(1000)

![](s2fig3.png)

## Visualizing the problem

![](s2fig4.png)

## Tobit Model

- The idea of the Tobit model is "model" not only why $y$ changes when $X$ changes, but also why y is censored.
  - Although you do that with the same parameters, under normality assumptions.

- When the data is censored, the likelihood function is similar to a probit model, when the data is not censored, the likelihood function is similar to a linear model:

$$\begin{aligned}
L_i(\beta,\sigma) &= \Phi\left(\frac{y^c-x_i'\beta}{\sigma}\right) \text{if } y_i = y^c \\
L_i(\beta,\sigma) &= \frac{1}{\sigma\sqrt{2\pi}} e^{ -\frac{1}{2}\left(\frac{y_i-x_i'\beta}{\sigma}\right)^2 } \text{if } y_i > y^c 
\end{aligned}
$$

## Estimation `Stata`

In `Stata`, you can estimate a Tobit model using the `tobit` command.

`tobit y x1 x2 x3, ll(#)` 

- `y`: dependent variable
- `x1 x2 x3`: independent variable
- `ll(#)`: is the value of the censoring point.

## Visualizing the solution


In [ ]:
*| echo: false
*| output: false
tobit y x, ll(0)
predict y_hat

two (scatter  y x, color(%50)) || (lfit y x) || (line yp x, sort lw(2) lc(gs2%20)), legend(off) name(m1, replace) title(Fully observed)

two (scatter  yc x, color(%50)) || (line y_hat x, sort) || (line yp x, sort lw(2) lc(gs2%20)), legend(off) name(m2, replace) title(Censored Data)

graph combine m1 m2, ycommon
graph export s2fig5.png, replace width(1000)

![](s2fig5.png)

# Tobit: Interpretation

## Latent Variable

- The easiest to interpret is the latent variable. 
- For example, say that you are interested in the effect of education on wages, but wages are censored at 10.
- In this case the coefficients of the Tobit model are the same as the coefficients of the linear model.

`tobit y x, ll(0)`

- use `margins` if you have interactions or polynomial terms.

## Data is Corner Solution:

- If data is corner solution, then you need to decide what to interpret.
  - For example, say you are interested in the effect of education hours of work
  - Hours of work cannot fall below 0.
  - But you know education has a positive effect (on something)

- Would you be interested in the effect on the probability of working?
- The effect on hours of work for those who work?
- The overall average effect on hours of work? (some will enter the labor force, some will work more hours)

## Probability of Working

$$P(y_i>0|x_i) = \Phi\left(\frac{x_i'\beta}{\sigma}\right)$$

`margins, dydx(x) predict(pr(0,.))`

- `predict(pr(0,.))` says you are interested in the probability that data was not censored...or in this case that was not a corner solution

## E(Y|Y>0,X) 

$$\begin{aligned}
y_i &= x_i'\beta + \epsilon_i \ || E(|y>0,X) \\
E(y_i|y_i>0,x_i) &= x_i'\beta + \sigma\lambda\left(\frac{x_i'\beta}{\sigma}\right) \\
\lambda(z) &= \frac{\phi(z)}{\Phi(z)}
\end{aligned}
$$

This is the expected value of the latent variable, conditional on the latent variable being positive.

`margins, dydx(x) predict(e(0,.))`

- `predict(e(0,.))` says you are interested in the expected change only for those who currently work.

## E(Y|X) 

$$\begin{aligned}
E(y_i|x_i) &= E(y_i|y_i>0,x_i) * P(y_i>0|x_i) + 0 * (1-P(y_i>0|x_i)) \\
E(y_i|x_i) &= \Phi\left(\frac{x_i'\beta}{\sigma}\right)
\left( x_i'\beta + \sigma\lambda\left(\frac{x_i'\beta}{\sigma}\right)\right) \\
E(y_i|x_i) &= \Phi\left(\frac{x_i'\beta}{\sigma}\right) x_i'\beta + \sigma \phi\left(\frac{x_i'\beta}{\sigma}\right)
\end{aligned}
$$ 

`margins, dydx(x) predict(ystar(0,.))`

- `predict(ystar(0,.))` says you are interested in the average effect considering those who work and those who do not work.

## Visualizing the solution


In [ ]:
*| echo: false
*| output: false
tobit y x, ll(0)
predict y_e, e(0,.) 
predict y_s, ystar(0,.)
scatter yc y_e y_s y_h x, ///
legend(order(1 "Censored Data" 2 "E(Y|Y>0)" 3 "E(Y)" 4 "Y_c"))
graph export s2fig6.png, replace width(1000)

![](s2fig6.png)

# Sample Selection: Heckman

## Exogenous Sample Selection

- First: Samples already represent a selection of the population.

  - however, because the selection is random, all assumptions of OLS are satisfied. (if they are true for the population.)
  
- Second: Some times selection may not be random, but based on observed (and control) characteristics

  - Not a problem either. Since you could at least say something for those you observe. (if you have the right variables)
  - This was exogenous sample selection.

ie: You want to estimate the effect of education on wages, but you only have data for highly educated people.

##


In [ ]:
*| echo: false
*| output: false
clear
set obs 1000
gen x = rnormal()
gen e = rnormal()
gen y = 1 + x + e
gen dpe = (x+.7*e+ rnormal()*.5)>.5
gen dp1 = normal(x)<runiform()

two (scatter y x, pstyle(p1)  msize(2) mlcolor(%0)  color(%30)) (lfit y x, pstyle(p1) lw(1)) ///
	(scatter y x if dp1, pstyle(p2) msize(2) mlcolor(%0) color(%30)) (lfit y x if dp1, pstyle(p2) lw(1)), ///
	legend(order(1 "Full" 3 "EX Subsample") pos(6) col(2)) ///
  xtitle("")

graph export s2fig7.png, replace width(1000)

![](s2fig7.png)

## Endogenous Sample Selection

- Third: Selection may be based on unobserved characteristics.

  - This is a problem. The reason why we do not observe data is for unknown reasons (part of the error).
  - Because we cannot control for it, it will bias our estimates. (like omitted variable bias)
  - This is endogenous sample selection.

ie: 
  - You want to estimate the effect of education on wages, but you only have data for those who work.
  - Those who work do so because they may have been offer higher wages for unknown reasons. (high skill? high motivation? )

##


In [ ]:
*| echo: false
*| output: false
two (scatter y x, pstyle(p1)  msize(2) mlcolor(%0)  color(%30)) (lfit y x, pstyle(p1) lw(1)) ///
	(scatter y x if dpe, pstyle(p2) msize(2) mlcolor(%0) color(%30)) (lfit y x if dpe, pstyle(p2) lw(1)), ///
	legend(order(1 "Full" 3 "End Subsample") pos(6) col(2)) ///
  xtitle("")

graph export s2fig8.png, replace width(1000)

![](s2fig8.png)

## Heckman Selection Model

- The Heckman selection model is an estimation method that allows you to correct a specific kind of endogenous sample selection.

- Consider the following model:

$$y_i = x_i'\beta + \epsilon_i$$

- In absence of selection, we can assume standard assumtions and estimate the model using OLS.

- However, if we have endogenous selection, usually means that we have a second equation that determines the selection.

$$s_i^* = z_i'\gamma + \eta_i$$


## 

The Full model:

$$\begin{aligned}
y_i &= x_i'\beta + \epsilon_i \text{ if } s_i^*>0 \\
s_i^* &= z_i'\gamma + \eta_i \\
\epsilon &\sim N(0,\sigma_\epsilon) \\
\eta &\sim N(0,1) \\
corr(\epsilon,\eta) &= \rho
\end{aligned}
$$ 

- $x_i$ and $z_i$ are not necessarily the same. But they are exogenous to the error terms.
- The selection equation depends on observable $z_i$ and unobservable $\eta_i$ factors.
- The unobserable factors are correlated with the error term of the main equation. 
- The problem: if $\rho\neq 0$ then $E(\epsilon|s_i^*>0,x_i) \neq 0$.

## Solution

- The solution is to "control" for the unobserved factors that are correlated with $\epsilon$. 

$$\begin{aligned}
y_i &= x_i'\beta + \epsilon_i \ || \ E( * |x_i,s_i^*>0) \\
E(y_i|x_i,s_i^*>0) &= x_i'\beta + E(\epsilon_i|x,z,\eta, s_i^*>0) \\
 &= x_i'\beta + E(\epsilon_i|\eta,s_i^*>0) \\
  &= x_i'\beta + \rho \frac{\phi(z_i'\gamma)}{\Phi(z_i'\gamma)} \\
  &= x_i'\beta + \rho \lambda (z_i'\gamma) \\
\end{aligned}
$$

Thus the new model is:

$$y_i  = x_i'\beta + \rho \lambda (z_i'\gamma) + \varepsilon_i$$

where $\gamma$ is estimated using a probit model.

## Implementation

- Two options:
  1. Estimate both outcome and selection equation jointly using MLE.
    - Requires careful setup of the likelihood function.
    - Imposes the assumption of joint normality of the error terms.
  2. Estimate it using a two-step procedure.(Heckit)
    - Estimate the selection equation using probit. $z_i'\gamma$
    - Estimate the outcome equation using OLS, inclusing inverse mills ratio. $\lambda (z_i'\gamma)$
    - Std Errs need to be corrected 

- Consideration: In contrast with IV, Heckman does not require an instrument, but having one is highly recommended.

## Example `Stata` {.scrollable}

Lets start by loading some data


In [ ]:
*| echo: true
*| code-fold: false
*| classes: larger
webuse womenwk, clear
describe

In `Stata`, we can use command `heckman` to estimate the Heckman selection model, but lets start by doing this manually


In [ ]:
*| echo: true
*| code-fold: false
*| classes: larger
gen works = (wage!=.)
** Selection model
probit works married children educ age
predict zg, xb

This selection equation can be interpreted the usual way

The outcome model:


In [ ]:
*| echo: true
*| code-fold: false
*| classes: larger
gen mill = normalden(zg)/normal(zg)
reg  wage educ age mill
est sto hkit

Lets compare the outcome with Stata's Heckman


In [ ]:
*| echo: true
*| code-fold: false
*| classes: larger
*| output: false
set linesize 255
reg wag educ age
est sto ols
heckman wage educ age, select(works = married children educ age) twostep
est sto hecktwo
heckman wage educ age, select(works = married children educ age) 
est sto heckmle

In [ ]:
*| echo: true
*| code-fold: false
*| classes: larger
esttab ols hkit hecktwo heckmle, se(%9.3f) b(%9.3f) star(* 0.10 ** 0.05 *** 0.01) nogaps mtitle(OLS Heckit Hkm-two Hkm-mle) nonum

## Interpretation

- It depends...

- but the most likely scenario is to interpret the outcomes for everyone (thus just look at coefficients of the outcome equation)

- But you can also obtain effects for those who work, or the average effect. 

- The Mills ratio can be interpreted as the direction of the selection.
  - If positive, then those who work are those who earn more
  - If negative, then those who work are those who earn less

## Extra example {.scrollable}


In [ ]:
*| echo: true
*| code-fold: false
*| classes: larger
*| output: false
frause oaxaca, clear
reg lnwage educ exper tenure 
est sto ols
heckman lnwage educ exper tenure  , select(lfp =educ age   married divorced kids6 kids714) 
est sto hk_mle
heckman lnwage educ exper tenure  , select(lfp =educ age   married divorced kids6 kids714) two
est sto hk_two

In [ ]:
*| echo: true
*| code-fold: false
*| classes: larger
esttab ols hk_mle hk_two, se(%9.3f) b(%9.3f) star(* 0.10 ** 0.05 *** 0.01) nogaps mtitle(OLS Heckit Hkm-two) nonum  

# The End...
Til next week